#SPY Spreadsheet

In [1]:
%matplotlib inline

Use future imports for python 3.0 forward compatibility

In [2]:
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division
from __future__ import absolute_import

Other imports

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime
from talib.abstract import *
import pinkfish as pf
import itable

# format price data
pd.options.display.float_format = '{:0,.2f}'.format

# Double the DPI, so we are making 2x plots:
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

Some global data

In [4]:
symbol = 'SPY'
capital = 10000
start = datetime.datetime(2000, 1, 1)
end = datetime.datetime.now()

In [5]:
from pandas.io.data import DataReader

def fetch_timeseries(symbol, dir_name='data', use_cache=True):
    """
    Read time series data. Use cached version if it exists and
    use_cache is True, otherwise retrive, cache, then read.
    """
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)    

    timeseries_cache = os.path.join(dir_name, symbol + '.csv')
    
    if os.path.isfile(timeseries_cache) and use_cache:
        pass
    else:
        ts = DataReader(symbol, 'yahoo', start=datetime.datetime(1900, 1, 1))
        ts.to_csv(timeseries_cache, encoding='utf-8')
    
    ts = pd.read_csv(timeseries_cache, index_col='Date', parse_dates=True)
    ts = _adj_column_names(ts)
    return ts

Fetch symbol data from internet; do not use local cache. 

In [6]:
ts = pf.fetch_timeseries(symbol, use_cache=False)

In [7]:
ts.tail()

,open,high,low,close,volume,adj_close
date,,,,,,
2015-06-01,211.94,212.34,210.62,211.57,92024700,211.57
2015-06-02,211.02,212.19,210.27,211.36,89527800,211.36
2015-06-03,212.00,212.67,211.33,211.92,86954600,211.92
2015-06-04,211.07,211.86,209.75,210.13,141934300,210.13
2015-06-05,209.95,210.58,208.98,209.77,113411500,209.77


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [8]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=True)

In [9]:
ts.head()

,open,high,low,close,volume,adj_close
date,,,,,,
1999-01-04,92.18,93.56,90.95,91.93,9450400,91.93
1999-01-05,91.86,93.31,91.86,92.98,8031000,92.98
1999-01-06,94.01,95.45,93.96,95.22,7737700,95.22
1999-01-07,94.43,95.06,93.98,94.75,5504900,94.75
1999-01-08,95.78,96.01,94.12,95.45,6224400,95.45


Add technical indicator: 200 day MA

In [10]:
sma200 = SMA(ts, timeperiod=200)
ts['sma200'] = sma200

In [11]:
ts.tail()

,open,high,low,close,volume,adj_close,sma200
date,,,,,,,
2015-06-01,211.94,212.34,210.62,211.57,92024700,211.57,203.05
2015-06-02,211.02,212.19,210.27,211.36,89527800,211.36,203.14
2015-06-03,212.00,212.67,211.33,211.92,86954600,211.92,203.24
2015-06-04,211.07,211.86,209.75,210.13,141934300,210.13,203.32
2015-06-05,209.95,210.58,208.98,209.77,113411500,209.77,203.39


Add technical indicator: ATR

In [12]:
atr = ATR(ts, timeperiod=14)
ts['atr'] = atr

In [13]:
ts.tail()

,open,high,low,close,volume,adj_close,sma200,atr
date,,,,,,,,
2015-06-01,211.94,212.34,210.62,211.57,92024700,211.57,203.05,1.74
2015-06-02,211.02,212.19,210.27,211.36,89527800,211.36,203.14,1.75
2015-06-03,212.00,212.67,211.33,211.92,86954600,211.92,203.24,1.72
2015-06-04,211.07,211.86,209.75,210.13,141934300,210.13,203.32,1.75
2015-06-05,209.95,210.58,208.98,209.77,113411500,209.77,203.39,1.74


Add technical indicator: 5 day high, and 5 day low

In [14]:
high5 = pd.rolling_max(ts.high, 5)
low5 = pd.rolling_min(ts.low, 5)
ts['high5'] = high5
ts['low5'] = low5

In [15]:
ts.tail()

,open,high,low,close,volume,adj_close,sma200,atr,high5,low5
date,,,,,,,,,,
2015-06-01,211.94,212.34,210.62,211.57,92024700,211.57,203.05,1.74,212.98,210.20
2015-06-02,211.02,212.19,210.27,211.36,89527800,211.36,203.14,1.75,212.98,210.27
2015-06-03,212.00,212.67,211.33,211.92,86954600,211.92,203.24,1.72,212.67,210.27
2015-06-04,211.07,211.86,209.75,210.13,141934300,210.13,203.32,1.75,212.67,209.75
2015-06-05,209.95,210.58,208.98,209.77,113411500,209.77,203.39,1.74,212.67,208.98


Add technical indicator: RSI, and 2-period cumulative RSI

In [16]:
rsi2 = RSI(ts, timeperiod=2)
ts['rsi2'] = rsi2

c2rsi2 = pd.rolling_sum(ts.rsi2, 2)
ts['c2rsi2'] = c2rsi2

In [17]:
ts.tail()

,open,high,low,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2
date,,,,,,,,,,,,
2015-06-01,211.94,212.34,210.62,211.57,92024700,211.57,203.05,1.74,212.98,210.20,44.02,66.97
2015-06-02,211.02,212.19,210.27,211.36,89527800,211.36,203.14,1.75,212.98,210.27,34.74,78.76
2015-06-03,212.00,212.67,211.33,211.92,86954600,211.92,203.24,1.72,212.67,210.27,69.27,104.01
2015-06-04,211.07,211.86,209.75,210.13,141934300,210.13,203.32,1.75,212.67,209.75,15.81,85.08
2015-06-05,209.95,210.58,208.98,209.77,113411500,209.77,203.39,1.74,212.67,208.98,12.06,27.87


Add technical indicator: Midpoint

In [18]:
mp = (ts.high + ts.low)/2
ts['mp'] = mp

In [19]:
ts.tail()

,open,high,low,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp
date,,,,,,,,,,,,,
2015-06-01,211.94,212.34,210.62,211.57,92024700,211.57,203.05,1.74,212.98,210.20,44.02,66.97,211.48
2015-06-02,211.02,212.19,210.27,211.36,89527800,211.36,203.14,1.75,212.98,210.27,34.74,78.76,211.23
2015-06-03,212.00,212.67,211.33,211.92,86954600,211.92,203.24,1.72,212.67,210.27,69.27,104.01,212.00
2015-06-04,211.07,211.86,209.75,210.13,141934300,210.13,203.32,1.75,212.67,209.75,15.81,85.08,210.81
2015-06-05,209.95,210.58,208.98,209.77,113411500,209.77,203.39,1.74,212.67,208.98,12.06,27.87,209.78


Add technical indicator: SMA10 of midpoint

In [20]:
sma10mp = pd.rolling_mean(ts.mp, 10)
ts['sma10mp'] = sma10mp

In [21]:
ts.head(10)

,open,high,low,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp
date,,,,,,,,,,,,,,
1999-01-04,92.18,93.56,90.95,91.93,9450400,91.93,nan,nan,nan,nan,nan,nan,92.25,nan
1999-01-05,91.86,93.31,91.86,92.98,8031000,92.98,nan,nan,nan,nan,nan,nan,92.58,nan
1999-01-06,94.01,95.45,93.96,95.22,7737700,95.22,nan,nan,nan,nan,100.00,nan,94.71,nan
1999-01-07,94.43,95.06,93.98,94.75,5504900,94.75,nan,nan,nan,nan,77.90,177.90,94.52,nan
1999-01-08,95.78,96.01,94.12,95.45,6224400,95.45,nan,nan,96.01,90.95,86.71,164.61,95.07,nan
1999-01-11,95.41,95.41,93.56,94.54,7578300,94.54,nan,nan,96.01,91.86,42.58,129.29,94.48,nan
1999-01-12,94.31,94.31,92.46,92.84,7768800,92.84,nan,nan,96.01,92.46,14.65,57.23,93.39,nan
1999-01-13,89.97,93.49,89.94,92.18,10810600,92.18,nan,nan,96.01,89.94,9.75,24.40,91.72,nan
1999-01-14,92.37,92.58,90.34,90.57,11400700,90.57,nan,nan,96.01,89.94,3.68,13.43,91.46,nan


Add technical indicator: Standard Deviation

In [22]:
sd = pd.rolling_std(ts.mp, 10)
ts['sd'] = sd

In [23]:
casey = ts['sd'] + 1
ts['casey'] = casey

In [24]:
ts.tail()

,open,high,low,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,casey
date,,,,,,,,,,,,,,,,
2015-06-01,211.94,212.34,210.62,211.57,92024700,211.57,203.05,1.74,212.98,210.20,44.02,66.97,211.48,212.40,0.74,1.74
2015-06-02,211.02,212.19,210.27,211.36,89527800,211.36,203.14,1.75,212.98,210.27,34.74,78.76,211.23,212.25,0.81,1.81
2015-06-03,212.00,212.67,211.33,211.92,86954600,211.92,203.24,1.72,212.67,210.27,69.27,104.01,212.00,212.14,0.75,1.75
2015-06-04,211.07,211.86,209.75,210.13,141934300,210.13,203.32,1.75,212.67,209.75,15.81,85.08,210.81,211.90,0.77,1.77
2015-06-05,209.95,210.58,208.98,209.77,113411500,209.77,203.39,1.74,212.67,208.98,12.06,27.87,209.78,211.57,0.90,1.90


In [25]:
upper = ts.sma10mp + ts.sd*2
lower = ts.sma10mp - ts.sd*2
ts['upper'] = upper
ts['lower'] = lower

In [26]:
ts.tail()

,open,high,low,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,casey,upper,lower
date,,,,,,,,,,,,,,,,,,
2015-06-01,211.94,212.34,210.62,211.57,92024700,211.57,203.05,1.74,212.98,210.20,44.02,66.97,211.48,212.40,0.74,1.74,213.89,210.92
2015-06-02,211.02,212.19,210.27,211.36,89527800,211.36,203.14,1.75,212.98,210.27,34.74,78.76,211.23,212.25,0.81,1.81,213.88,210.62
2015-06-03,212.00,212.67,211.33,211.92,86954600,211.92,203.24,1.72,212.67,210.27,69.27,104.01,212.00,212.14,0.75,1.75,213.65,210.63
2015-06-04,211.07,211.86,209.75,210.13,141934300,210.13,203.32,1.75,212.67,209.75,15.81,85.08,210.81,211.90,0.77,1.77,213.44,210.36
2015-06-05,209.95,210.58,208.98,209.77,113411500,209.77,203.39,1.74,212.67,208.98,12.06,27.87,209.78,211.57,0.90,1.90,213.36,209.78


Select a smaller time from for use with itable

In [27]:
df = ts['2015-01-01':]

In [28]:
df.head()

,open,high,low,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,casey,upper,lower
date,,,,,,,,,,,,,,,,,,
2015-01-02,205.46,205.96,203.27,204.52,121465900,204.52,193.20,2.28,208.04,203.27,6.95,14.45,204.62,206.47,0.96,1.96,208.40,204.54
2015-01-05,203.26,203.46,200.46,200.82,169632600,200.82,193.29,2.41,208.04,200.46,1.19,8.14,201.96,206.03,1.73,2.73,209.48,202.58
2015-01-06,201.19,201.82,197.98,198.93,209151400,198.93,193.37,2.51,207.44,197.98,0.64,1.83,199.90,205.47,2.60,3.60,210.67,200.26
2015-01-07,200.52,201.82,199.99,201.41,125346700,201.41,193.47,2.54,207.26,197.98,54.91,55.56,200.90,204.95,2.96,3.96,210.87,199.03
2015-01-08,203.10,205.24,203.08,204.98,147217800,204.98,193.58,2.63,205.96,197.98,82.49,137.41,204.16,204.68,2.89,3.89,210.45,198.91


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [29]:
pt = itable.PrettyTable(df, tstyle=itable.TableStyle(theme="theme1"), center=True, header_row=True, rpt_header=20)

pt.update_col_header_style(format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] == 'volume':
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high5'][row] == pt.df['high'][row]) and \
       (pt.df['high5'][row] > pt.df['high'][row-1]):
        col = df.columns.get_loc("high5")    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low5'][row] == pt.df['low'][row]) and \
       (pt.df['low5'][row] < pt.df['low'][row-1]):
        col = df.columns.get_loc("low5")
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')          

In [30]:
pt

,OPEN,HIGH,LOW,CLOSE,VOLUME,ADJ_CLOSE,SMA200,ATR,HIGH5,LOW5,RSI2,C2RSI2,MP,SMA10MP,SD,CASEY,UPPER,LOWER
2015/01/02,205.46,205.96,203.27,204.52,121465900,204.52,193.20,2.28,208.04,203.27,6.95,14.45,204.62,206.47,0.96,1.96,208.40,204.54
2015/01/05,203.26,203.46,200.46,200.82,169632600,200.82,193.29,2.41,208.04,200.46,1.19,8.14,201.96,206.03,1.73,2.73,209.48,202.58
2015/01/06,201.19,201.82,197.98,198.93,209151400,198.93,193.37,2.51,207.44,197.98,0.64,1.83,199.90,205.47,2.60,3.60,210.67,200.26
2015/01/07,200.52,201.82,199.99,201.41,125346700,201.41,193.47,2.54,207.26,197.98,54.91,55.56,200.90,204.95,2.96,3.96,210.87,199.03
2015/01/08,203.10,205.24,203.08,204.98,147217800,204.98,193.58,2.63,205.96,197.98,82.49,137.41,204.16,204.68,2.89,3.89,210.45,198.91
2015/01/09,205.48,205.50,202.61,203.34,158567300,203.34,193.69,2.65,205.50,197.98,52.80,135.29,204.05,204.37,2.76,3.76,209.89,198.86
2015/01/12,203.50,203.69,201.02,201.75,144396100,201.75,193.79,2.65,205.50,197.98,31.10,83.90,202.36,203.85,2.57,3.57,208.98,198.72
2015/01/13,203.21,204.57,199.62,201.18,214553300,201.18,193.89,2.81,205.50,199.62,24.05,55.15,202.09,203.29,2.23,3.23,207.76,198.82
2015/01/14,198.76,200.21,197.69,199.97,192991100,199.97,193.97,2.86,205.50,197.69,12.21,36.26,198.95,202.49,2.20,3.20,206.88,198.09
2015/01/15,200.73,201.11,198.00,198.14,176613900,198.14,194.04,2.88,205.50,197.69,4.91,17.13,199.55,201.85,2.02,3.02,205.89,197.82
